In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *


from PIL import Image
from numpy import array
import cv2

In [3]:
fileNum = 1
seg_layer = 0
X = 0
Y = 1

vector_num = 0
vectors_array = []
one_image = []

# Vector Conversion: cartesian-to-polar
- Insert vectors into a list

In [4]:
# vectors[vector index][segmentation layer][x or y coordinate]


# This loop controls the file number 1-21
while fileNum < 22:
    
    vectors = np.load(f'vectorList{fileNum}.npy')
    vect_length = len(vectors)
    
    vect_index = 0
    
    # This loop controls which vector is being converted, ~998-1000 vectors per file
    while vect_index < vect_length:
        
        seg_layer = 0
        one_image.clear()  
        
        # This loop iterates through all segmentation layers for ONE image
        while seg_layer < 32:
            
            polar = cv2.cartToPolar(vectors[vect_index][seg_layer][X], 
                            vectors[vect_index][seg_layer][Y])
            
            one_image.append(polar)
            seg_layer += 1
        
        
        vectors_array.insert(0, one_image)
        vect_index += 1
    
    fileNum += 1
    print('file no. : ', fileNum)


file no. :  2
file no. :  3
file no. :  4
file no. :  5
file no. :  6
file no. :  7
file no. :  8
file no. :  9
file no. :  10
file no. :  11
file no. :  12
file no. :  13
file no. :  14
file no. :  15
file no. :  16
file no. :  17
file no. :  18
file no. :  19
file no. :  20
file no. :  21
file no. :  22


In [5]:
len(vectors_array)

20399

In [6]:
vectors_array.reverse()

In [7]:
vectors = array(vectors_array)
type(vectors)

numpy.ndarray

In [8]:
type(vectors_array)

list

# Load Labels (speeds)

In [9]:
labels = np.loadtxt('train.txt')
len(labels)

20400

In [10]:
type(labels)

numpy.ndarray

# Split up data (training set, valid set, test set)

In [46]:
train_idx = array(vectors_array[:14279])
valid_idx = array(vectors_array[14280:18360])
test_idx = array(vectors_array[18360:])

# Prepare labels for each set

In [47]:
trainLabels = array(labels[:14279])
validLabels = array(labels[14280:18360])
testLabels = array(labels[18360:])

In [49]:
train = TensorDataset(torch.from_numpy(train_idx), torch.from_numpy(trainLabels))
valid = TensorDataset(torch.from_numpy(valid_idx), torch.from_numpy(validLabels))

# test = TensorDataset(torch.from_numpy(test_idx), torch.from_numpy(testLabels))

In [53]:
len(train_idx)

14279

# Lstm Class

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [67]:
class LSTM(nn.Module):
    
    # input = 32 bc there's 32 features
    # hidden_layer = ??
    # output size = 1, bc we want to predict the speed for one frame
    
    def __init__(self, input_size=32, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [69]:
model = LSTM()

### Set loss function, set learning rate

In [72]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)

In [73]:
print(model)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


### Set number of epochs, train the model

In [74]:
epochs = 10
# epochs = 150

for i in range(epochs):
    
    for seq, labels in train_inout_seq:
        optimzer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size), 
                            torch.zeros(1, 1, model.hidden_layer_size))
        
        y_pred = model(seq)
        
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
    
    if i % 25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item(): 10.8f}')
        
print(f'epoch: {i:3} loss: {single_loss.item(): 10.10f}')

NameError: name 'train_inout_seq' is not defined